In [1]:
import jax
import numpy as np
from scipy.stats.contingency import association
import pandas as pd

In [17]:
a = np.random.randint(0, 5, size = 500000)
b = np.random.randint(0, 3, size = 500000)

In [18]:
def xtab_counter(jax_xtab, x_y):
    x, y = x_y
    jax_xtab = jax_xtab.at[x, y].add(1)
    return jax_xtab, None

In [19]:
xtab_counter_cpu = jax.jit(xtab_counter, backend='cpu')

In [20]:
%%timeit
# get the shape of the contingency table
cols = a, b
uniq_vals_all_cols, idx = zip( *(np.unique(col, return_inverse=True) for col in cols) )
shape_xt = [uniq_vals_col.size for uniq_vals_col in uniq_vals_all_cols]

jax_xt = jax.numpy.zeros(shape_xt, dtype=np.int32)
xtab_result, _ = jax.lax.scan(xtab_counter, init = jax_xt, xs = (idx[0], idx[1]))
association(xtab_result, method='cramer')

4.62 s ± 87.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [21]:
%%timeit
cols = a, b
uniq_vals_all_cols, idx = zip( *(np.unique(col, return_inverse=True) for col in cols) )
shape_xt = [uniq_vals_col.size for uniq_vals_col in uniq_vals_all_cols]

jax_xt = jax.numpy.zeros(shape_xt, dtype=np.int32)
xtab_result, _ = jax.lax.scan(xtab_counter_cpu, init = jax_xt, xs = (idx[0], idx[1]))
association(xtab_result, method='cramer')

45.6 ms ± 856 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [22]:
%%timeit
# numpy crosstab
cols = a, b
uniq_vals_all_cols, idx = zip( *(np.unique(col, return_inverse=True) for col in cols) )
shape_xt = [uniq_vals_col.size for uniq_vals_col in uniq_vals_all_cols]

xt = np.zeros(shape_xt, dtype=np.int32)
np.add.at(xt, idx, wt)
association(xt, method='cramer')

88 ms ± 8.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [23]:
%%timeit
xtab_result = pd.crosstab(a, b)
association(xtab_result, method='cramer')

58.5 ms ± 8.68 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
